In [24]:
import numpy as np
import cv2
# Mantener el background frame para quitarlo posteriormente
background = None
# Guarda los datos de la mano para que todos sus detalles estén en un solo lugar.
hand = None
# Variables para contar cuántos fotogramas han pasado y para establecer el tamaño de la ventana.
frames_elapsed = 0
FRAME_HEIGHT = 300
FRAME_WIDTH = 400
# Prueba a editarlas si tu programa tiene problemas para reconocer tu tono de piel.
CALIBRATION_TIME = 80
BG_WEIGHT = 0.1
OBJ_THRESHOLD = 30

In [26]:
class HandData:
    # Inicialización de variables para almacenar la información geométrica y el estado de la mano
    top = (0,0)
    bottom = (0,0)
    left = (0,0)
    right = (0,0)
    centerX = 0  # Posición central de la mano en el eje X
    prevCenterX = 0  # Posición X anterior para detectar movimientos
    isInFrame = False  # Indica si la mano está en el marco
    isWaving = False  # Indica si la mano está haciendo un gesto de movimiento
    fingers = None  # Almacena el número de dedos detectados
    gestureList = []  # Lista para almacenar los gestos detectados temporalmente

    def __init__(self, top, bottom, left, right, centerX):
        # Constructor que inicializa las propiedades de la mano con valores específicos
        self.top = top
        self.bottom = bottom
        self.left = left
        self.right = right
        self.centerX = centerX
        self.prevCenterX = 0  # Se inicia en cero para futuras comparaciones
        self.isInFrame = False  # Se asume que la mano no está en el marco inicialmente
        self.isWaving = False  # Se asume que no hay movimiento inicialmente

    def update(self, top, bottom, left, right):
        # Método para actualizar la posición de la mano en el marco
        self.top = top
        self.bottom = bottom
        self.left = left
        self.right = right

    def check_for_waving(self, centerX):
        # Método para verificar si la mano está haciendo un gesto de movimiento (onda)
        self.prevCenterX = self.centerX  # Guarda la posición X actual como previa
        self.centerX = centerX  # Actualiza la posición central X con el nuevo valor
        # Comprueba si el movimiento entre las dos posiciones X es mayor que 3
        if abs(self.centerX - self.prevCenterX) > 3:
            self.isWaving = True  # Si es así, se considera que la mano está ondeando
        else:
            self.isWaving = False  # De lo contrario, no hay movimiento significativo

In [28]:
def write_on_image(frame):
    """
    Actualiza la imagen capturada con información textual sobre el estado de detección de la mano
    y dibuja un rectángulo alrededor de la región de interés.

    Args:
    frame: El frame actual obtenido de la cámara.

    La función verifica el estado de la detección:
    - Si está en periodo de calibración, muestra 'Calibrando...'.
    - Si no se detecta la mano, muestra 'Mano no detectada'.
    - Si se detecta la mano y está realizando un movimiento, muestra 'Moviendo'.
    - Muestra el número de dedos detectados si la mano está estática.
    """
    # Inicializa el texto a mostrar en función del estado de la mano.
    text = "Buscando..."
    if frames_elapsed < CALIBRATION_TIME:
        text = "Calibrando..."
    elif hand is None or not hand.isInFrame:
        text = "Mano no detectada"
    else:
        if hand.isWaving:
            text = "Moviendo"
        elif hand.fingers == 0:
            text = "Cero"
        elif hand.fingers == 1:
            text = "Uno"
        elif hand.fingers == 2:
            text = "Dos"

    # Dibuja el texto en el frame dos veces para crear un efecto de borde que mejore la legibilidad.
    cv2.putText(frame, text, (10, 20), cv2.FONT_HERSHEY_COMPLEX, 0.4, (0, 0, 0), 2)
    cv2.putText(frame, text, (10, 20), cv2.FONT_HERSHEY_COMPLEX, 0.4, (255, 255, 255), 1)

    # Dibuja un rectángulo alrededor de la región de interés para indicar dónde debe colocarse la mano.
    cv2.rectangle(frame, (region_left, region_top), (region_right, region_bottom), (255, 255, 255), 2)


In [20]:
def get_region(frame):
    # Esta línea extrae una subsección del marco original basada en las coordenadas predefinidas.
    # La región extraída es donde se espera encontrar o analizar la mano o cualquier objeto de interés.
    region = frame[region_top:region_bottom, region_left:region_right]

    # Convertir la región de interés a escala de grises facilita el procesamiento de imagen siguiente,
    # porque reduce la complejidad de la imagen al eliminar la información de color,
    # dejando solo la intensidad de los píxeles que es más útil para la detección de bordes.
    region = cv2.cvtColor(region, cv2.COLOR_BGR2GRAY)

    # Aplicar un filtro gaussiano ayuda a suavizar la imagen, lo que reduce el ruido y las variaciones de intensidad.
    # Esto es particularmente útil para preparar la imagen para procesos de detección de bordes y contornos,
    # ya que evita que se detecten falsos positivos causados por pequeñas imperfecciones o ruido en la imagen.
    region = cv2.GaussianBlur(region, (5,5), 0)

    # Devuelve la región de interés procesada y lista para la siguiente etapa de procesamiento de imagen.
    return region

In [16]:
def get_average(region):
    """
    Actualiza el fondo para sustracción de fondo utilizando un promedio ponderado.
    Esta técnica ayuda a identificar y segmentar objetos en movimiento en la imagen.

    Args:
    region (ndarray): La región actual de interés en la imagen capturada de la cámara.

    La función verifica si ya se ha establecido un fondo. Si no es así, inicializa el fondo
    con la región actual. Si el fondo ya existe, actualiza este fondo con un promedio ponderado
    de las imágenes actuales, lo que permite adaptar el fondo a los cambios graduales en la
    escena como cambios en la iluminación o en la configuración del entorno.
    """

    # Utilizamos la variable global 'background' para mantener el estado del fondo a través de las llamadas a la función
    global background

    # Si 'background' no está inicializado, lo establecemos con la región actual
    if background is None:
        # Copia la región a 'background' y convierte a tipo float para cálculos futuros
        background = region.copy().astype("float")
        return  # Finaliza la función después de inicializar el fondo

    # Si el fondo ya está inicializado, actualiza el fondo con un promedio ponderado
    # Esto permite que el fondo se ajuste a cambios en la escena
    cv2.accumulateWeighted(region, background, BG_WEIGHT)

In [34]:
def segment(region):
    """
    Segmenta la mano del fondo en una región de interés (ROI) mediante diferenciación y umbralización.
    Devuelve la región segmentada y el contorno de la mano si se detecta alguna.

    Args:
    region (array): La ROI del frame actual en escala de grises.
    """
    global hand  # Referencia al objeto global que representa la mano detectada.

    # Calcula la diferencia absoluta entre el fondo promediado y el frame actual para resaltar cambios.
    diff = cv2.absdiff(background.astype(np.uint8), region)

    # Aplica un umbral binario para convertir la diferencia en una imagen binaria donde los valores altos indican movimiento.
    thresholded_region = cv2.threshold(diff, OBJ_THRESHOLD, 255, cv2.THRESH_BINARY)[1]

    # Encuentra los contornos en la imagen umbralizada. Los contornos son útiles para identificar formas como la mano.
    contours, _ = cv2.findContours(thresholded_region.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Si no se encuentran contornos, se considera que no hay mano en la región.
    if len(contours) == 0:
        if hand is not None:
            hand.isInFrame = False  # Actualiza el estado de la mano a no detectada.
        return  # Sale de la función sin retornar objetos.

    # Si se encuentran contornos, se selecciona el mayor contorno que probablemente sea la mano.
    else:
        if hand is not None:
            hand.isInFrame = True  # Actualiza el estado de la mano a detectada.
        segmented_region = max(contours, key=cv2.contourArea)  # Elige el contorno de área máxima.

        # Devuelve la imagen umbralizada junto con el contorno más grande encontrado.
        return (thresholded_region, segmented_region)

In [36]:
def get_hand_data(thresholded_image, segmented_image):
    global hand  # Utiliza la variable 'hand' definida globalmente para almacenar datos de la mano

    # Utiliza Convex Hull para encontrar el contorno que conecta todos los puntos extremos de la mano
    convexHull = cv2.convexHull(segmented_image)

    # Extrae las coordenadas extremas del convex hull para definir los límites de la mano
    top = tuple(convexHull[convexHull[:, :, 1].argmin()][0])
    bottom = tuple(convexHull[convexHull[:, :, 1].argmax()][0])
    left = tuple(convexHull[convexHull[:, :, 0].argmin()][0])
    right = tuple(convexHull[convexHull[:, :, 0].argmax()][0])

    # Calcula el centro de la mano para ayudar en la detección de movimientos y la localización de dedos
    centerX = int((left[0] + right[0]) / 2)

    # Si 'hand' no está inicializada, crea una nueva instancia de HandData con los valores obtenidos
    if hand == None:
        hand = HandData(top, bottom, left, right, centerX)
    else:
        # Si ya está inicializada, actualiza la información de la mano
        hand.update(top, bottom, left, right)

    # Verifica si la mano está ondeando, una vez cada 6 fotogramas para reducir la sensibilidad
    if frames_elapsed % 6 == 0:
        hand.check_for_waving(centerX)

    # Añade el recuento de dedos a una lista de gestos para promediar el resultado
    hand.gestureList.append(count_fingers(thresholded_image))
    # Determina el número de dedos más comúnmente identificado cada 12 fotogramas
    if frames_elapsed % 12 == 0:
        hand.fingers = most_frequent(hand.gestureList)
        hand.gestureList.clear()  # Limpia la lista de gestos para el próximo ciclo de conteo

In [38]:
def count_fingers(thresholded_image):
    # Determinar la altura en la imagen donde se contará los dedos, basado en la posición superior e inferior de la mano detectada.
    line_height = int(hand.top[1] + (0.2 * (hand.bottom[1] - hand.top[1])))

    # Crear una imagen binaria donde se trazará la línea para contar los dedos.
    line = np.zeros(thresholded_image.shape[:2], dtype=int)

    # Dibujar una línea horizontal a la altura calculada que abarque todo el ancho de la imagen.
    cv2.line(line, (thresholded_image.shape[1], line_height), (0, line_height), 255, 1)

    # Realizar una operación AND a nivel de bit para resaltar donde la línea intersecta con los dedos en la imagen umbralizada.
    line = cv2.bitwise_and(thresholded_image, thresholded_image, mask=line.astype(np.uint8))

    # Detectar los contornos en la línea donde se intersectan los dedos, interpretando cada sección continua como un dedo.
    contours, _ = cv2.findContours(line.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    # Inicializar contador de dedos.
    fingers = 0

    # Contar los dedos asegurando que cada contorno detectado no sea demasiado ancho para ser un dedo.
    for curr in contours:
        width = len(curr)
        # Confirmar que el contorno es lo suficientemente delgado para ser considerado un dedo.
        if width < 3 * abs(hand.right[0] - hand.left[0]) / 4 and width > 5:
            fingers += 1

    # Devolver el número total de dedos detectados.
    return fingers

In [40]:
def most_frequent(input_list):
    # Diccionario para contar la frecuencia de cada elemento en la lista
    freq_dict = {}
    count = 0  # Variable para almacenar la frecuencia máxima actual
    most_freq = 0  # Variable para almacenar el elemento más frecuente

    # Itera sobre la lista en orden inverso para priorizar el último valor más frecuente en caso de empates
    for item in reversed(input_list):
        # Incrementa el conteo del elemento en el diccionario
        freq_dict[item] = freq_dict.get(item, 0) + 1

        # Si el conteo del elemento actual es mayor o igual al mayor conteo registrado
        if freq_dict[item] >= count:
            count = freq_dict[item]  # Actualiza el mayor conteo
            most_freq = item  # Actualiza el elemento más frecuente

    # Devuelve el elemento que más frecuentemente aparece en la lista
    return most_freq


In [42]:
import cv2

# Definición de la región de interés en la parte superior derecha del frame
region_top = 0
region_bottom = int(2 * FRAME_HEIGHT / 3)
region_left = int(FRAME_WIDTH / 2)
region_right = FRAME_WIDTH

frames_elapsed = 0  # Contador de fotogramas para gestionar la calibración
capture = cv2.VideoCapture(0)  # Inicia la captura de vídeo con la cámara predeterminada

# Bucle infinito para el procesamiento de cada frame capturado
while True:
    ret, frame = capture.read()  # Captura un frame de la cámara
    if not ret:
        break  # Si no hay frame, rompe el bucle

    # Redimensiona el frame al tamaño especificado y lo voltea horizontalmente
    frame = cv2.resize(frame, (FRAME_WIDTH, FRAME_HEIGHT))
    frame = cv2.flip(frame, 1)  # Voltea el frame para simular un efecto espejo

    # Obtiene la región de interés preparada para la detección de bordes
    region = get_region(frame)

    # Proceso de calibración y segmentación
    if frames_elapsed < CALIBRATION_TIME:
        get_average(region)  # Calcula el promedio del fondo durante el tiempo de calibración
    else:
        # Segmenta la mano del fondo una vez finalizada la calibración
        region_pair = segment(region)
        if region_pair is not None:
            # Si la segmentación fue exitosa, muestra la región segmentada
            thresholded_region, segmented_region = region_pair
            cv2.drawContours(region, [segmented_region], -1, (255, 255, 255))
            cv2.imshow("Segmented Image", region)  # Muestra la imagen segmentada

            # Actualiza los datos de la mano basados en la imagen segmentada
            get_hand_data(thresholded_region, segmented_region)

    # Actualiza el frame con la información de la mano y muestra el resultado
    write_on_image(frame)
    cv2.imshow("Camera Input", frame)  # Muestra el frame en la ventana

    frames_elapsed += 1  # Incrementa el contador de fotogramas

    # Permite salir del bucle con la tecla 'x'
    if cv2.waitKey(1) & 0xFF == ord('c'):
        break

# Limpieza: libera la captura de la cámara y cierra todas las ventanas abiertas
capture.release()
cv2.destroyAllWindows()

Codigo Completo, Funcional para 2 dedos y Moviendo

In [45]:
import numpy as np
import cv2
import serial
import time

# Configuración inicial de la comunicación serial
ser = serial.Serial('COM11', 9600, timeout=2)  # Ajusta el nombre del puerto y la tasa de baudios
time.sleep(3)  # Espera para que la conexión se establezca

# Dimensiones y regiones de la imagen
FRAME_HEIGHT = 300
FRAME_WIDTH = 400
region_top = 0
region_bottom = int(2 * FRAME_HEIGHT / 3)
region_left = int(FRAME_WIDTH / 2)
region_right = FRAME_WIDTH

# Configuraciones de procesamiento de imagen
background = None
hand = None
frames_elapsed = 0
CALIBRATION_TIME = 80
BG_WEIGHT = 0.2
OBJ_THRESHOLD = 30

class HandData:
    # Inicialización de variables para almacenar la información geométrica y el estado de la mano
    def __init__(self, top, bottom, left, right, centerX):
        self.top = top
        self.bottom = bottom
        self.left = left
        self.right = right
        self.centerX = centerX
        self.prevCenterX = 0
        self.isInFrame = False
        self.isWaving = False
        self.fingers = None
        self.gestureList = []

    def update(self, top, bottom, left, right):
        self.top = top
        self.bottom = bottom
        self.left = left
        self.right = right

    def check_for_waving(self, centerX):
        self.prevCenterX = self.centerX
        self.centerX = centerX
        if abs(self.centerX - self.prevCenterX) > 3:
            self.isWaving = True
            ser.write(b'3')
        else:
            self.isWaving = False

def write_on_image(frame):
    text = "Buscando..."
    if frames_elapsed < CALIBRATION_TIME:
        text = "Calibrando..."
    elif not hand or not hand.isInFrame:
        text = "Mano no detectada"
    else:
        if hand.isWaving:
            text = "Moviendo"
        elif hand.fingers is not None:
            if hand.fingers == 0:
                text = "Cero"
                ser.write(b'0')
            elif hand.fingers == 1:
                text = "Uno"
                ser.write(b'1')
            elif hand.fingers == 2:
                text = "Dos"
                ser.write(b'2')

    cv2.putText(frame, text, (10, 20), cv2.FONT_HERSHEY_COMPLEX, 0.4, (0, 0, 0), 2)
    cv2.putText(frame, text, (10, 20), cv2.FONT_HERSHEY_COMPLEX, 0.4, (255, 255, 255), 1)
    cv2.rectangle(frame, (region_left, region_top), (region_right, region_bottom), (255, 255, 255), 2)


def get_region(frame):
    region = frame[region_top:region_bottom, region_left:region_right]
    region = cv2.cvtColor(region, cv2.COLOR_BGR2GRAY)
    region = cv2.GaussianBlur(region, (5,5), 0)
    return region

def get_average(region):
    global background
    if background is None:
        background = region.copy().astype("float")
    else:
        cv2.accumulateWeighted(region, background, BG_WEIGHT)

def segment(region):
    global hand
    diff = cv2.absdiff(cv2.convertScaleAbs(background), region)
    thresholded_region = cv2.threshold(diff, OBJ_THRESHOLD, 255, cv2.THRESH_BINARY)[1]
    contours, _ = cv2.findContours(thresholded_region.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        if hand is not None:
            hand.isInFrame = False
        return None
    max_contour = max(contours, key=cv2.contourArea)
    if hand is not None:
        hand.isInFrame = True
    return thresholded_region, max_contour

def get_hand_data(thresholded_image, segmented_image):
    global hand
    convexHull = cv2.convexHull(segmented_image)
    top = tuple(convexHull[convexHull[:, :, 1].argmin()][0])
    bottom = tuple(convexHull[convexHull[:, :, 1].argmax()][0])
    left = tuple(convexHull[convexHull[:, :, 0].argmin()][0])
    right = tuple(convexHull[convexHull[:, :, 0].argmax()][0])
    centerX = int((left[0] + right[0]) / 2)
    if hand is None:
        hand = HandData(top, bottom, left, right, centerX)
    else:
        hand.update(top, bottom, left, right)
    if frames_elapsed % 6 == 0:
        hand.check_for_waving(centerX)
    hand.gestureList.append(count_fingers(thresholded_image))
    if frames_elapsed % 12 == 0:
        hand.fingers = most_frequent(hand.gestureList)
        hand.gestureList.clear()

def count_fingers(thresholded_image):
    line_height = int(hand.top[1] + (0.2 * (hand.bottom[1] - hand.top[1])))
    line = np.zeros(thresholded_image.shape[:2], dtype=int)
    cv2.line(line, (thresholded_image.shape[1], line_height), (0, line_height), 255, 1)
    line = cv2.bitwise_and(thresholded_image, thresholded_image, mask=line.astype(np.uint8))
    contours, _ = cv2.findContours(line.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    fingers = 0
    for curr in contours:
        width = len(curr)
        if width < 3 * abs(hand.right[0] - hand.left[0]) / 4 and width > 5:
            fingers += 1
    return fingers

def most_frequent(input_list):
    freq_dict = {}
    count = 0
    most_freq = 0
    for item in reversed(input_list):
        freq_dict[item] = freq_dict.get(item, 0) + 1
        if freq_dict[item] >= count:
            count = freq_dict[item]
            most_freq = item
    return most_freq

capture = cv2.VideoCapture(0)  # Asegúrate de que el índice de la cámara es correcto

while True:
    ret, frame = capture.read()
    if not ret:
        break
    frame = cv2.resize(frame, (FRAME_WIDTH, FRAME_HEIGHT))
    frame = cv2.flip(frame, 1)
    region = get_region(frame)
    if frames_elapsed < CALIBRATION_TIME:
        get_average(region)
    else:
        result = segment(region)
        if result:
            thresholded_region, segmented_region = result
            get_hand_data(thresholded_region, segmented_region)
    write_on_image(frame)
    cv2.imshow("Camera Input", frame)
    frames_elapsed += 1
    if cv2.waitKey(1) & 0xFF == ord('c'):
        break

capture.release()
cv2.destroyAllWindows()
ser.close()  # Cierra el puerto serial al finalizar

SerialException: WriteFile failed (PermissionError(13, 'El dispositivo no reconoce el comando.', None, 22))

In [43]:
capture.release()
cv2.destroyAllWindows()
ser.close()  # Cierra el puerto serial al finalizar